In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
date = ['2021.09.01 - 2022.01.01', '2021.06.01 - 2021.09.01', '2021.03.01 - 2021.06.01', '2021.01.01 - 2021.03.01']
date2 = ['2020.09.01 - 2021.01.01', '2020.06.01 - 2020.09.01', '2020.03.01 - 2020.06.01', '2020.01.01 - 2020.03.01']
date3 = ['2019.09.01 - 2020.01.01', '2019.06.01 - 2019.09.01', '2019.03.01 - 2019.06.01', '2019.01.01 - 2019.03.01']
date4 = ['2018.09.01 - 2019.01.01', '2018.06.01 - 2018.09.01', '2018.03.01 - 2018.06.01', '2018.01.01 - 2018.03.01']
date5 = ['2017.09.01 - 2018.01.01', '2017.06.01 - 2017.09.01', '2017.03.01 - 2017.06.01', '2017.01.01 - 2017.03.01']
date6 = ['2016.09.01 - 2017.01.01', '2016.06.01 - 2016.09.01', '2016.03.01 - 2016.06.01', '2016.01.01 - 2016.03.01']
date7 = ['2015.09.01 - 2016.01.01', '2015.06.01 - 2015.09.01', '2015.03.01 - 2015.06.01', '2015.01.01 - 2015.03.01']
date8 = ['2014.09.01 - 2015.01.01', '2014.06.01 - 2014.09.01', '2014.03.01 - 2014.06.01', '2014.01.01 - 2014.03.01']
date9 = ['2013.09.01 - 2014.01.01', '2013.06.01 - 2013.09.01', '2013.03.01 - 2013.06.01', '2013.01.01 - 2013.03.01']
date10 = ['2012.09.01 - 2013.01.01', '2012.06.01 - 2012.09.01', '2012.03.01 - 2012.06.01', '2012.01.01 - 2012.03.01']
date11 = ['2011.09.01 - 2012.01.01', '2011.06.01 - 2011.09.01', '2011.03.01 - 2011.06.01', '2011.01.01 - 2011.03.01']
date12 = ['2010.09.01 - 2011.01.01', '2010.06.01 - 2010.09.01', '2010.03.01 - 2010.06.01', '2010.01.01 - 2010.03.01']
date = date + date2 + date3 + date4 + date5 + date6 + date7 + date8 + date9 + date10 + date11 + date12

date.reverse()

In [5]:
path = '/Volumes/L7aler_HD 1/ADL_Project/AR_data/mag_flux_area_data/set_2/'
for i, name in enumerate(date[1:]):
    try:
        if i == 0:
            df = pd.read_csv(path + name + '.csv', index_col = 0)
        else:
            temp = pd.read_csv(path + name + '.csv', index_col = 0)
            df = pd.concat([df, temp])
    except:
        pass
    
df = df.set_index([pd.Index(np.arange(0, len(df), 1))])

In [6]:
Id = df['HARPNUM'].unique()

path_input = '/Volumes/L7aler_HD 1/ADL_Project/AR_data/mag_flux_area_data/set_2/input_sequences/'
path_target = '/Volumes/L7aler_HD 1/ADL_Project/AR_data/mag_flux_area_data/set_2/target_sequences/'


data_in_conc = np.zeros(shape = (100, 2))
target_conc = np.zeros(shape = (25, 2))

c = 0
for i, harpnum in enumerate(Id):
    l, u = 0, 100   #l is the lower index for the input, u is the upper index for the input (so initially input is elements 0-100)
    j = np.where(df['HARPNUM'] == harpnum)[0]
    flux = df['USFLUX'][j]
    area = df['AREA_ACR'][j]

    sequ = 0
    while u + 25 < len(j):
        data_in = np.zeros(shape = (100, 4))
        target = np.zeros(shape = (25, 4))
        data_in[:, 0], data_in[:, 1] = flux[l:u], area[l:u]
        target[:, 0], target[:, 1] = flux[u:u+25], area[u:u+25]
        
        
        l, u = l + 10, u + 10    #we shift our kernel window by 10 date points for each new sequence
        sequ += 1
        
        name_in = 'i_AR{}_{}.npz'.format(harpnum, sequ)
        name_target = 't_AR{}_{}.npz'.format(harpnum, sequ)
        #with open(path_input + name_in, 'wb') as f:
        #    np.save(f, data_in)
        #with open(path_target + name_target, 'wb') as f:
        #    np.save(f, target)
        
        if c == 0:
            c += 1
            data_in_conc = data_in
            target_conc = target
        elif c ==1:
            c+= 1
            data_in_conc = np.concatenate(([data_in_conc], [data_in]))
            target_conc = np.concatenate(([target_conc], [target]))
        else:
            data_in_conc = np.concatenate((data_in_conc, [data_in]))
            target_conc = np.concatenate((target_conc, [target]))
    

    break
        
    

In [7]:
len(data_in_conc)

83

### If you want to downsample the data

In [ ]:
import spectres

In [ ]:
data = np.copy(data_in_conc)   #I'm annoyed by this long variable name
new_wavs = np.arange(1, 100, 3)
spec_wavs = np.arange(0, 100, 1)
data_res = np.zeros(shape=(len(data), 33, 2))
for i, d in enumerate(data):
    data_res[i, :, 0] = spectres.spectres(new_wavs, spec_wavs, d[:, 0], spec_errs=None, fill=None, verbose=True)
    data_res[i, :, 1] = spectres.spectres(new_wavs, spec_wavs, d[:, 1], spec_errs=None, fill=None, verbose=True)

In [ ]:
target = np.copy(target_conc)
new_wavs = np.arange(1, 25, 3)
spec_wavs = np.arange(0, 25, 1)
target_res = np.zeros(shape=(79566, 8, 2))
for i, d in enumerate(target):
    target_res[i, :, 0] = spectres.spectres(new_wavs, spec_wavs, d[:, 0], spec_errs=None, fill=None, verbose=True)
    target_res[i, :, 1] = spectres.spectres(new_wavs, spec_wavs, d[:, 1], spec_errs=None, fill=None, verbose=True)

In [ ]:
name = '/Volumes/L7aler_HD 1/ADL_Project/AR_data/mag_flux_area_data/set_3/input_sequences/input_res_params_concatenated.npy'
with open(name, 'wb') as f:
    #np.save(f, data_res)
    #np.save(f, data_in_conc)
name = '/Volumes/L7aler_HD 1/ADL_Project/AR_data/mag_flux_area_data/set_3/target_sequences/target_res_params_concatenated.npy'
with open(name, 'wb') as f:
    #np.save(f, target_res)
    #np.save(f, target_conc)